# Notebook 2_a

In [2]:
import pickle
from pathlib import Path
import tdt
import pandas as pd 
import papermill as pm

In [3]:
import Notebook_2_helper_functions as Nb_2

In [4]:
mouse = 'T240'
data_day = '2019_09_27__T240'
date = '2019_09_27'

photo_day = 'T240-145905'

HowManyBack=1 

data_dir_input = "/Users/gilmandelbaum/Desktop/T240_mouse"
data_dir_output = "/Users/gilmandelbaum/Desktop/output/"

# Import behavior related data per state transition

In [5]:
behavior_data_states = pd.read_csv(data_dir_input+"/"+mouse+"/"+data_day+"/"+mouse+"_dataAnalog.csv")

In [6]:
behavior_data_states = Nb_2.prepare_behavior_data(behavior_data_states)

# Import photometry data


In [7]:
photometry_data = tdt.read_block(data_dir_input+"/"+mouse+"/"+photo_day)

read from t=0s to t=1345.46s


# Grab only what is needed from the TDT photo tanks and make a df. 

In [8]:
#The photometry TDT data structure has a lot of info. 
#We will create 1 data frame that has: 
# 
#['d2 R' , 'd1 R', 'd2 L', "d1 L", "toBehSys", "fromBehSys" ]

def reading_in_photometryData_all_Channels(data):
    photometry_all_channels_d = {}
    photometry_all_channels_d["d2 R"]=photometry_data.streams.grn1.data 
    photometry_all_channels_d["d1 R"]=photometry_data.streams.red1.data 
    
    
    photometry_all_channels_d["d2 L"]=photometry_data.streams.grn2.data 
    photometry_all_channels_d["d1 L"]=photometry_data.streams.red2.data 
    
    
    
    photometry_all_channels_d["toBehSys"]=photometry_data.streams.toGG.data 
    photometry_all_channels_d["fromBehSys"]=photometry_data.streams.froG.data 
    photometry_all_channels= pd.DataFrame(data=photometry_all_channels_d)
    
    return (photometry_all_channels)

In [9]:
photometry_all_channels=reading_in_photometryData_all_Channels(photometry_data)

# Clean the photometry data (or any neural data)

In [10]:
photometry_all_channels_clean = Nb_2.handshake_behav_recording_sys(photometry_all_channels)

# Clean behavior states data 

In [11]:
behavior_data_states_clean = Nb_2.define_behavior_data_states(behavior_data_states)

# pre syncing steps for photometry data (or any neural data) and behavior data 

In [12]:
behavior_data_states_clean_binNumberlist=Nb_2.number_of_bins_beh_data_per_trial(behavior_data_states_clean)

In [13]:
photometry_all_channels_clean_binNumberlist = Nb_2.number_of_bins_photo_data_per_trial(photometry_all_channels_clean)

# Resample the photometry data (or any neural data) and align to 200 hz state transition behavioral data 

In [14]:
aligned_behav_photo = Nb_2.resample_photoData_and_align_photo_with_beh(behavior_data_states_clean_binNumberlist,
                                                            behavior_data_states_clean,
                                                            photometry_all_channels_clean,
                                                            photometry_all_channels_clean_binNumberlist)

number of trials synced: 269


In [15]:
aligned_behav_photo = Nb_2.tag_real_full_trial(aligned_behav_photo)

/Users/gilmandelbaum/Desktop/photometry_analysis/Notebook_2_helper_functions.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prevTrial['new_index'] = df_prevTrial.index+1


In [16]:
aligned_behav_photo.to_csv("aligned_behav_photo.csv")

In [17]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_2_a.pickle"
my_path = root / d 
my_file = open(my_path, 'wb')
my_file = pickle.dump(aligned_behav_photo,my_file)